# Titanic - Gradient_boosting

## Trainning model

In [3]:
from os.path import join
import sys
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from tuningModel import tune_gradient_boosting
# === Load dữ liệu ===
path_dir = join("..", "..", "..")
input_dir = join(path_dir, "data", "feature_engineered", "scaledData")

df_train = pd.read_csv(join(input_dir, "scaledData_engineered_train.csv"))
df_test = pd.read_csv(join(input_dir, "scaledData_engineered_test.csv"))

X = df_train.drop(['Survived'], axis=1)
y = df_train['Survived']
best_model, results_df = tune_gradient_boosting(X,y)
# === KFold ===
kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies, precisions, recalls, f1s = [], [], [], []

fold_index = 1
for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # === Mô hình Gradient Boosting ===
    model = best_model

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    # === Tính metric ===
    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)

    print(f"Fold {fold_index} - Acc: {acc:.4f} | F1: {f1:.4f}")
    fold_index += 1

# === Mean và Std ===
mean_acc = np.mean(accuracies)
mean_prec = np.mean(precisions)
mean_rec = np.mean(recalls)
mean_f1 = np.mean(f1s)
std_acc = np.std(accuracies)

print("\n==== Mean metrics ====")
print(f"Accuracy: {mean_acc:.4f}")
print(f"Precision: {mean_prec:.4f}")
print(f"Recall: {mean_rec:.4f}")
print(f"F1-score: {mean_f1:.4f}")
print(f"Std (Accuracy): {std_acc:.4f}")


Fitting 5 folds for each of 972 candidates, totalling 4860 fits

 Best Score (accuracy): 0.8485
 Best Params: {'learning_rate': 0.05, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300, 'subsample': 0.6}
Fold 1 - Acc: 0.8380 | F1: 0.8027
Fold 2 - Acc: 0.8034 | F1: 0.7059
Fold 3 - Acc: 0.8539 | F1: 0.8088
Fold 4 - Acc: 0.7753 | F1: 0.6825
Fold 5 - Acc: 0.8427 | F1: 0.7879

==== Mean metrics ====
Accuracy: 0.8227
Precision: 0.7884
Recall: 0.7323
F1-score: 0.7576
Std (Accuracy): 0.0291


## Save model into logger and dump model to pickle file

In [5]:
import os
import sys
import joblib
import pandas as pd
from os.path import join
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

# === Thêm đường dẫn để import log_experiment ===
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from log.experiment_logger import log_experiment

# === Ghi log kết quả vào CSV ===
log_path = join(path_dir, "log", "experiment_log.csv")
log_experiment(
    output_path=log_path,
    model_name="GradientBoostingClassifier",
    feature_name="scaledData_engineered",
    params=best_model.get_params(),
    kfold=5,
    f1=mean_f1,
    acc=mean_acc,
    rec=mean_rec,
    prec=mean_prec,
    std=std_acc,
    author="Thang"
)

# === Huấn luyện lại trên toàn bộ dữ liệu train ===
final_model = best_model
final_model.fit(X, y)

# === Dump model ra .pkl ===
model_dir = join(path_dir, "notebook", "model", "gradient_boosting", "Model Pickles")
os.makedirs(model_dir, exist_ok=True)
model_path = join(model_dir, "gb_scaledData_tuned.pkl")
joblib.dump(final_model, model_path)
print(f"✅ Model saved to {model_path}")
df_original = pd.read_csv(join(path_dir,"data","raw","test.csv"))
passenger_ids = df_original["PassengerId"]
# === Tạo file submission ===
X_test = df_test.copy()
if 'Survived' in X_test.columns:
    X_test = X_test.drop(columns=['Survived'])

y_test_pred = final_model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,  # đảm bảo test có cột này
    'Survived': y_test_pred
})

sub_dir = join(path_dir, "notebook", "model", "gradient_boosting", "submissions")
os.makedirs(sub_dir, exist_ok=True)
submission_path = join(sub_dir, "submission_gb_scaledData_tuned.csv")
submission.to_csv(submission_path, index=False)
print(f"📤 Submission file saved to {submission_path}")

Logged experiment to ..\..\..\log\experiment_log.csv
✅ Model saved to ..\..\..\notebook\model\gradient_boosting\Model Pickles\gb_scaledData_tuned.pkl
📤 Submission file saved to ..\..\..\notebook\model\gradient_boosting\submissions\submission_gb_scaledData_tuned.csv


# The end